<a href="https://colab.research.google.com/github/BrunoGallani/projeto_PLN_UFABC/blob/main/Case_RAG_Enriquecido_Credito_Carbono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl ifconfig.me

34.186.61.52

# Base para desenvolvimento

In [ ]:
!pip install chromadb pypdf sentence-transformers umap-learn helper_utils

In [ ]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
import umap
from google import genai
from google.colab import userdata

In [ ]:
chave_gemini = userdata.get("GEMINI_KEY")
cliente = genai.Client(api_key=chave_gemini)

# Leitura/Tratamento Fonte Informação

In [ ]:
faq_mercado_carbono=PdfReader("/content/mercado_regulado_carbono_cvm.pdf")


In [ ]:
texto_faq_mercado_carbono= [pagina.extract_text().strip() for pagina in faq_mercado_carbono.pages]

In [ ]:
refinado_faq_mercado_carbono = [texto for texto in texto_faq_mercado_carbono if texto.strip()!=""]

# Alocação de Token/Embedding

In [ ]:
divide_caracteres = RecursiveCharacterTextSplitter(separators=["\n\n","\n",". "," ",""], chunk_size =1000, chunk_overlap = 0 )

In [ ]:
texto_caracteres_divididos = divide_caracteres.split_text("\n\n".join(refinado_faq_mercado_carbono))

In [ ]:
texto_caracteres_divididos[10]

'cumprimento dos compromissos internacionais do Brasil, especialmente aqueles \nassumidos no Acordo de Paris . Ao estabelecer um mercado de carbono \nregulado, o país cria um mecanismo econômico para atingir as metas de redução \nde emissões de GEE para 2035, previstas na Contribuição Nacionalmente \nDeterminada (NDC), que estabelece uma redução de 59% a 67% em \ncomparação a indicadores de 2005 4, rumo a um modelo de desenvolvimento \nsustentável. \n12 - Como o mercado voluntário se relaciona com o SBCE? \nDe acordo com a Lei n° 15.042/2024, o mercado voluntário  é o ambiente \ncaracterizado por transações de créditos de carbono ou ativos integrantes do \nSBCE de forma espontânea , onde os participantes negociam esses créditos \npara compensar suas próprias emissões sem uma obrigação legal específica  e \nsem gerar ajustes na contabilidade nacional de emissões. \nJá as entidades sujeitas ao cumprimento de limites de emissão estabelecidos'

In [ ]:
divide_token = SentenceTransformersTokenTextSplitter(chunk_overlap=20, tokens_per_chunk=256)

In [ ]:
texto_token = []
for texto in texto_caracteres_divididos:
  texto_token += divide_token.split_text(texto)

In [ ]:
len(texto_token)

22

In [ ]:
texto_token

['r set / 2025 gefis / soi faq – mercado regulado de carbono',
 'faq – mercado regulado de carbono ( set / 2025 ) 2 faq – mercado regulado de carbono 1 – o que e o sistema brasileiro de comercio de emissoes de gases de efeito estufa ( sbce )? o sistema brasileiro de comercio de emissoes de gases de efeito estufa ( sbce ) e um ambiente regulado que funciona sob um regime de controle das emissoes de gases de efeito estufa ( gee ) e viabiliza a negociacao de ativos que representam o direito de emissao, bem como a reducao ou remocao de gee. ele atuara como uma ferramenta para reduzir as emissoes de gee, com vistas a promover o desenvolvimento sustentavel, a equidade climatica e estimular inovacoes tecnologicas de baixo carbono. o sbce funcionara no modelo de " cap - and - trade " : e estabelecido um limit',
 'no modelo de " cap - and - trade " : e estabelecido um limite total ( o " cap " ) de emissoes para cada setor ou empresa, os quais podem comprar e vender entre si ( o “ trade ” ) os a

In [ ]:
embedding_function = SentenceTransformerEmbeddingFunction()

## Criação do db

In [ ]:
banco_dados_vetorial_client = chromadb.Client()
colecao_chroma_carbono = banco_dados_vetorial_client.create_collection("mercado-carbono", embedding_function=embedding_function,)

In [ ]:
ids = [str(i) for i in range(len(texto_token))]

In [ ]:
colecao_chroma_carbono.add(ids=ids, documents=texto_token)

In [ ]:
colecao_chroma_carbono.count()

22

## Teste retrieval db

In [ ]:
query = "O mercado de carbono é bom?"

In [ ]:
resultado_query = colecao_chroma_carbono.query(query_texts=[query], n_results = 5)

In [ ]:
resultado_query["documents"][0]

['##do e o publico informados sobre o desenvolvimento do mercado de carbono no brasil. faq – mercado regulado de carbono ( set / 2025 ) 7',
 'estamos atentos aos desafios e as particularidades desse novo mercado. acompanharemos a criacao do novo orgao gestor e contribuiremos para a criacao de um arcabouco regulatorio robusto. alem disso, como e de praxe, a autarquia mantera o dialogo continuo com o mercado e a sociedade visando a construcao e ao aprimoramento das normas.',
 'os ativos a partir do momento que passam a ser negociados em mercados organizados. da mesma forma, os creditos de carbono, do mercado voluntario, tambem sao valores mobiliarios quando negociados no mercado financeiro e de capitais, conforme art. 2, inciso x da lei n° 6. 385 / 762. 5 - qual o papel da cvm nesse novo mercado? o art. 14 da lei n° 15. 042 / 2024 diz que “ os ativos integrantes do sbce e os creditos de carbono, quando negociados no mercado financeiro e de capitais, sao valores mobiliarios sujeitos ao re

# Enriquecimento de Prompt

In [ ]:
def gerar_multiplas_queries(query, modelo="gemini-2.0-flash"):
  prompt = """
    Você é um analista senior de finanças aplicado ao mercado de carbono, os clientes de sua empresa querem entender sobre o ramo.
    Para o cenário dado, proponha até cinco perguntas que ajude os clientes a achar a informação que precisam.
    Retorne questões concisas, em topicos simples e de escopo abrangente, evitando alocar mais de um assunto no mesmo tópico.
    Garanta que cada questão esteja completa e diretamente relacionada ao cenario dado.
    Questões separadas apenas por \n no final de cada pergunta e não devem ter numeração e nenhum outro marcador no começo da pergunta.
          """
  mensagens = prompt + "Consulta do Usuário: "+ query
  resposta = cliente.models.generate_content(
    model=modelo,
    contents = mensagens
  )
  return resposta

In [ ]:
query = "Mercado Carbono"
resultado = gerar_multiplas_queries(query)
original_e_augmented = [query] + resultado.candidates[0].content.parts[0].text.split("\n")
original_e_augmented_refinado = set()
for resultado in original_e_augmented:
  tratado = resultado.replace("*","").strip()
  if len(tratado) >0:
    original_e_augmented_refinado.add(tratado)

In [ ]:
original_e_augmented_refinado

{'Como a transparência e a rastreabilidade dos créditos de carbono podem ser aprimoradas para garantir a integridade do mercado e evitar o "greenwashing"?',
 'Como as políticas governamentais (nacionais e internacionais) e os acordos climáticos influenciam a demanda e a oferta de créditos de carbono?',
 'Mercado Carbono',
 'Quais são as estratégias de empresas para reduzir sua exposição ao risco de carbono e como o mercado de carbono pode ser utilizado como ferramenta de gestão de riscos?',
 'Quais são os principais mecanismos de precificação de carbono (taxas, mercados de licenças) e como eles impactam diferentes setores da economia?',
 'Quais são os riscos e oportunidades de investimento em projetos de crédito de carbono, considerando diferentes metodologias de certificação e a volatilidade dos preços?'}

In [ ]:
documentos_rag= colecao_chroma_carbono.query(query_texts=list(original_e_augmented_refinado), n_results = 5)["documents"]

In [ ]:
documentos_unicos = set()
for documentos in documentos_rag:
  for documento in documentos:
    documentos_unicos.add(documento)

In [ ]:
documentos_unicos

{'##do e o publico informados sobre o desenvolvimento do mercado de carbono no brasil. faq – mercado regulado de carbono ( set / 2025 ) 7',
 'cumprimento dos compromissos internacionais do brasil, especialmente aqueles assumidos no acordo de paris. ao estabelecer um mercado de carbono regulado, o pais cria um mecanismo economico para atingir as metas de reducao de emissoes de gee para 2035, previstas na contribuicao nacionalmente determinada ( ndc ), que estabelece uma reducao de 59 % a 67 % em comparacao a indicadores de 2005 4, rumo a um modelo de desenvolvimento sustentavel. 12 - como o mercado voluntario se relaciona com o sbce? de acordo com a lei n° 15. 042 / 2024, o mercado voluntario e o ambiente caracterizado por transacoes de creditos de carbono ou ativos integrantes do sbce de forma espontanea, onde os participantes negociam esses creditos para compensar suas proprias emissoes se',
 'estamos atentos aos desafios e as particularidades desse novo mercado. acompanharemos a cria